In [20]:
import pandas as pd
import torch as th
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2

from solution.dstl_detection.dstl_dataset import DstlDataset

ImportError: cannot import name 'DstlProcessing' from 'solution.dstl_detection.dstl_processing' (/Users/cristianion/Desktop/satimg_model/solution/dstl_detection/dstl_processing.py)

In [2]:
MODEL_NAME = "dstl_model"
LOCATION = "models/dstl"
DSTL_PATH = "/Users/cristianion/Desktop/satimg_data/DSTL"
TRAIN_WKT_FILE = DSTL_PATH + "/train_wkt_v4.csv"
GRID_SIZES_FILE = DSTL_PATH + "/grid_sizes.csv"

IMAGES_MULTICHANNEL = DSTL_PATH + "/sixteen_band"
IMAGES_RGB = DSTL_PATH + "/three_band"

# Column names
COL_MULTIPOLYGONWKT = "MultipolygonWKT"
COL_CLASSTYPE = "ClassType"
COL_IMAGEID = "ImageId"
COL_XMAX = "Xmax"
COL_YMIN = "Ymin"

# Image extension
EXT_TIFF = ".tif"


# resize parameters
IMAGE_RES_X = 512
IMAGE_RES_Y = 512


CLASSES = ["building", "structures", "road", "track", "tree", "crops", "waterway", "standing_water", "vehicle_large", "vehicle_small"]


In [7]:
DSTL_TRAIN_TRANSFORM = A.Compose([
    A.Resize(height=IMAGE_RES_Y, width=IMAGE_RES_X),
    A.Normalize(
        mean=[0.0, 0.0, 0.0],
        std=[1.0, 1.0, 1.0],
        max_pixel_value=255.0,
    ),
    ToTensorV2(),
])

DSTL_VAL_TRANSFORM = A.Compose([
    A.Resize(height=IMAGE_RES_Y, width=IMAGE_RES_X),
    A.Normalize(
        mean=[0.0, 0.0, 0.0],
        std=[1.0, 1.0, 1.0],
        max_pixel_value=255.0,
    ),
    ToTensorV2(),
])

In [8]:
dataset = DstlDataset(DSTL_TRAIN_TRANSFORM, train_file=TRAIN_WKT_FILE, grid_sizes_file=GRID_SIZES_FILE, classes=CLASSES, train_res_x=IMAGE_RES_X, train_res_y=IMAGE_RES_Y)

In [10]:
image, mask = dataset[0]

In [16]:
print("Image shape " + str(image.shape))
print("Mask shape " + str(mask.shape))

Image shape torch.Size([3, 512, 512])
Mask shape (512, 512)


In [18]:
# find CUDA / MPS / CPU device
device = (
    "cuda"
    if th.cuda.is_available()
    else "mps"
    if th.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device
